In [0]:
####
#### If you encounter an error, run this cell again.
####
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import logging
logging.getLogger('tensorflow').disabled = True

from google.colab import files
try:
    uploaded = files.upload()
except:
    uploaded = files.upload()

if len(uploaded.keys()) > 0:
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
        model = tf.keras.models.load_model(fn)
    model.summary()

    !wget --no-check-certificate --show-progress -q\
        "https://www.dropbox.com/s/y0q7lt3mq436es1/chest_xray_150x150.zip?dl=0" \
        -O "/tmp/chest_xray_150x150.zip"
    import zipfile
    import os
    zip_ref = zipfile.ZipFile("/tmp/chest_xray_150x150.zip", 'r')
    zip_ref.extractall("/tmp")
    zip_ref.close()
    root_dir = "/tmp/chest_xray_150x150"
    class_names = os.listdir(root_dir + "/train")

    ttest = {}
    ttest_files = {}
    for class_name in class_names:
        ttest_files[class_name] = os.listdir(root_dir + "/test/" + class_name)
    ttest['files'] = ttest_files
    ttest['gen'] = ImageDataGenerator(preprocessing_function=lambda x: x/127.5-1., rotation_range=3.14)
    ttest['dir'] = root_dir + "/test"
    ttest['flow'] = ttest['gen'].flow_from_directory(ttest['dir'], class_mode="binary", target_size=model.input_shape[1:3])
    score = model.evaluate(ttest['flow'])
    print('\nTest Loss: {:.3f}, Test Accuracy: {:.2f}'.format(score[0], score[1]))